In [1]:
# setup paths
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

# CREATE label map

In [2]:
# create label map - add all sign language glosses in sequential order

labels = [
    # phoenix sample glosses
    {'name':'montag', 'id':1},    
    {'name':'auch', 'id':2},  
    {'name':'mehr', 'id':3},  
    {'name':'wolke', 'id':4},  
    {'name':'als', 'id':5},  
    {'name':'sonne', 'id':6},  
    {'name':'ueberwiegend', 'id':7},  
    {'name':'regen', 'id':8},  
    {'name':'gewitter', 'id':9},  
    ]

with open(ANNOTATION_PATH + '/label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item {\n') # space removed
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# CREATE tfrecord file

In [3]:
# downloading pretrained tf models from tf model zoo
!cd Tensorflow && git clone https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 99321, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 99321 (delta 78), reused 52 (delta 44), pack-reused 99183 (from 5)
Receiving objects: 100% (99321/99321), 622.99 MiB | 30.48 MiB/s, done.
Resolving deltas: 100% (72155/72155), done.


In [ ]:
# installing object detection module (below)

In [9]:
%%bash
cd Tensorflow/models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf1/setup.py . # updated to tf1 
python -m pip install .

Processing /Users/maximilianscheel/Desktop/github/Sign2Voice_tensorflow/slr_tf_rtod/Tensorflow/models/research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for object_detection: filename=object_detection-0.1-py3-none-any.whl size=1660077 sha256=e50d49b65211f32046ae494396225617049d34e907d872e7d30a40a1f6ffc15c
  Stored in directory: /private/var/folders/1r/wl0hpmfs0x5_s1k9l77c0d6m0000gn/T/pip-ephem-wheel-cache-to6ifevk/wheels/cc/f4/a8/c7882bb8121d9bc87f9405645c017ff87d23697191a5380a6d
Successfully built object_detection
  Attempting uninstall: object_detection
    Found existing installation: object_detection 0.1
    Uninstalling object_detection-0.1:
      Successfully uninstalled object_detection-0.1



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [10]:
# update line of code in .venv/lib/python3.10/site-packages/official/vision/image_classification/augment.py

"""
replace
from tensorflow.python.keras.layers.preprocessing import image_preprocessing as image_ops 
with
from tensorflow.keras.preprocessing import image as image_ops
"""


import os

# Define the path to the Python file
file_path = "../.venv/lib/python3.10/site-packages/official/vision/image_classification/augment.py"

# Define the lines to be replaced
old_line = "from tensorflow.python.keras.layers.preprocessing import image_preprocessing as image_ops"
new_line = "from tensorflow.keras.preprocessing import image as image_ops"

# Read the file and copy the old line as a comment, then add the new line
with open(file_path, 'r') as file:
    lines = file.readlines()

with open(file_path, 'w') as file:
    for line in lines:
        if old_line in line:
            file.write("# " + line)  # Comment out the old line
            file.write(new_line + '\n')  # Add the new line
        else:
            file.write(line)

print(f"Updated {file_path} with new import statement and commented out the old one.")

Updated ../.venv/lib/python3.10/site-packages/official/vision/image_classification/augment.py with new import statement and commented out the old one.


In [11]:
# update protobuf before running code
!pip install --upgrade protobuf



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [12]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


In [13]:
# copy model config to training folder
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' # model rerun with augmented training data - folder Tensorflow/models/my_ssd_mobnet needs to be deleted otherwise training model script will throw an error

In [14]:
!mkdir -p {'Tensorflow/workspace/models/' + CUSTOM_MODEL_NAME}
!cp {PRETRAINED_MODEL_PATH + '/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH + '/' + CUSTOM_MODEL_NAME}

# UPDATE config for transfer learning

In [15]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [16]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [17]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [18]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [19]:
pipeline_config.model.ssd.num_classes = 9 # no. of phoenix sample glosses
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [20]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# TRAIN the model

In [21]:
# no. of steps set to 10000
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=10000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=10000


In [ ]:
# create ckpt checkpoint files in my_ssd_mobnet later used in streamlit python script to load the pretrained model
!python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=10000